# 합성곱 신경망(CNN)

- 합성곱 신경망 (Convolutional Neural Network, CNN)은 이미지 인식과 음성 인식등 다양한 곳에서 사용됨 

## 전체 구조

1) 이전의 신경망과 같이 계층을 조합하여 만들 수 있음
2) 합성곱계층(Convolutional layer)과 풀링계층(Pooling layer)이 등장
3) 완전연결층(fully conneted) : 인접하는 계층의 모든 뉴런과 결합된 신경망
4) CNN의 구조 (합성곱과 풀링계층 추가)

### 완전연결 계층(Affine 계층)으로 이뤄진 네트워크 예

- 여기서 ReLU는 Sigmoid로 대체 가능

<img src='./1.JPG'>

### CNN으로 이뤄진 네트워크 예

<img src='./2.JPG'>

- 새로운 합성곱 계층과 풀링 계층이 추가된다.
- CNN의 계층은 Conv-ReLU-(Pooling)의 흐름으로 연결된다 (풀링이 생략되기도 한다)
- Affine-ReLU가 Conv-ReLU-(Pooling)으로 바뀌었다.
- 또 새로운 점은 기존의 Affine-ReLU 계층을 그대로 사용할 수 있고, 마지막에서는 Softmax 함수를 활용해서 결과값을 얻을 수도 있다는 것이다.

## 합성곱 계층

- CNN에서는 패딩 (padding), 스트라이드 (stride) 등의 용어가 등장한다.
- 계층 사이에서는 3차원 데이터와 같이 입체적인 데이터가 흐른다는 점에서 기존의 신경망과 다르다.

### 완전연결 계층의 문제점

1) 완전연결 계층의 문제점은 무엇일까?
- 데이터의 형상이 무시된다는 사실
- 입력 데이터가 이미지인 경우, 가로, 세로, 채널(색상)으로 구성된 3차원 정보이다.
- 그러나 완전 연결 계층에 입력할 때에는 3차원 데이터를 1차원 데이터로 플랫하게 해주어야한다.
- 지금까지 MNIST 데이터셋이 (1, 28, 28)인 이미지를 784개의 데이터로 1렬로 만들어서 입력을 넣었다.
- 3차원 이미지 데이터는 인접한 픽셀의 위치 정보 등 3차원 속에서 의미를 갖는 본질적인 패턴이 존재할 수 있다.

2) 합성곱 계층은 형상을 유지한다.
- 데이터를 3차원으로 입력받아서 마찬가지로 다음 계층으로 3차원 데이터로 전달한다.
- 데이터를 제대로 이해할 가능성이 있게 된다.

3) 합성곱 계층의 입출력 데이터를 특징 맵 (feature map)이라고 한다. 입력을 입력 특징 맵, 출력을 출력 특징 맵이라고 하는 식이다.

### 합성곱 연산

- 이미지 처리에서 말하는 필터 연산에 해당
- 예시 : 입력데이터는 세로,가로 방향의 형상가짐 (필터도 마찬가지)
- 입력은 (4,4), 필터는 (3,3), 출력은(2,2)의 형상
- 문헌에 따라 필터를 커널이라고 지칭
- 합성곱 연산은 필터의 윈도우를 일정 간격 이동해가며 입력데이터에 적용
- 입력과 필터에서 대응하는 원소끼리 곱한 후 그 총합 계산 (단일 곱셈-누산, FMA)

<img src='./3.JPG'>

위 그림과 같이 합성곱 연산은 입력 데이터에 필터를 적용하며, 데이터와 필터의 형상을 (높이, 너비)로 표기한다.

<img src='./4.JPG'>

합성곱 연산은 필터의 윈도우(window)를 일정 간격으로 이동해가며 입력 데이터에 적용한다. 먼저 위 그림과 같이 입력 데이터와 필터에서 대응하는 원소끼리 곱한 후에 그 총합을 구한다. 그리고 나서 그 결과를 출력의 해당 장소에 저장하는 방식으로 연산이 이루어진다.

CNN에서는 필터의 매개변수가 앞서 공부해 온 "가중치"에 해당한다.
또한 CNN에서 편향까지 고려한다면 아래와 같이 합성곱 연산을 수행하게 된다.

<img src='./5.JPG'>

### 패딩

- 합성곱 연산을 수행하기 전 입력 데이터 주변을 특정 값 (ex.0)으로 채우는 것
- 예시 : (4,4)크기의 입력데이터에 폭이 1인 패딩 적용 

<img src='./6.JPG'>

1) (4,4) 입력데이터에 패딩 추가되어 (6,6)이 됨

2) 패딩은 출력크기를 조정할 목적으로 사용
- 연산을 거칠 때 마다 출력의 형상은 계속해서 줄어들음
- 출력 크기가 1이 되어버리면 합성곱 연산 불가
- 입력 데이터의 공간적 크기를 고정한 채 다음 계층에 전달 가능

### 스트라이드

- 필터를 적용하는 위치의 간격
- 스트라이드를 2로 하면 필터 적용 윈도우가 두 칸씩 이동

<img src='./7.JPG'>

1) 스트라이드를 키우면 출력크기는 작아짐

2) 패딩, 스트라이드, 출력크기의 관계 수식화
- 입력크기(H,W) / 필터크기(FH, FW) / 출력크기(OH, OW) / 패딩 P / 스트라이드 S
<img src='./8.JPG'>

3) 수식들은 정수로 나눠 떨어지는 값이어야 함

### 3차원 데이터의 합성곱 연산

- 이미지 데이터는 "세로 * 가로 * 채널" 모두를 고려한 3차원 데이터
- 2차원 데이터와 다르게 3차원 데이터는 채널 수가 "1"이 아님. 이 때문에 아래와 같이 입력 데이터와 필터의 합성곱 연산을 채널마다 수행하고, 그 결과를 더해서 하나의 출력을 얻는 식으로 진행.
- 여기서 주의할 점은 입력 데이터의 채널 수와 필터의 채널 수는 동일해야 한다는 것

<img src='./9.JPG'>

### 블록으로 생각하기

- 3차원 데이터의 경우, 앞서 본 2차원 데이터와 달리 직관적으로 이해하기가 조금 어려울 수 있다. 그러나 3차원 데이터의 합성곱 연산 시, 아래와 같이 데이터와 필터를 직육면체 블록이라고 생각하면 이해하기가 조금 수월함

<img src='./10.JPG'>

위 예시에서 출력 데이터는 한 장의 특징 맵(Feature Map)이다. 그렇다면 만약 합성곱 연산의 출력으로 다수의 채널을 내보내고자 한다면 어떻게 해야 할까? 그럴 때는 아래 그림과 같이 다수의 필터("가중치"를 의미)를 사용하면 됨

<img src='./11.JPG'>

여기에 "편향"까지 고려하고 싶다면 다음과 같이 합성곱 연산을 수행할 수 있음]

<img src='./12.JPG'>

이처럼 합성곱 연산에서는 "필터의 수"도 고려를 해주어야 한다. 필터의 가중치 데이터는 4차원 데이터이며, (출력 채널 수, 입력 채널 수, 높이, 너비) 순으로 쓴다.

앞서 공부할 때, 신경망 처리에서는 입력 데이터를 한 덩어리로 묶어서 "배치"로 처리했다.
 

이와 마찬가지로 합성곱 연산도 "배치" 처리를 수행해 줄 수 있다. 이를 위해 다음과 같이 각 계층을 흐르는 데이터의 차원을 하나 늘려서 4차원 데이터로 저장하겠다. 구체적으로는 데이터를 (데이터 수, 채널 수, 높이, 너비) 순으로 저장한다.

<img src='./13.JPG'>

- 여기서 주의할 점은 신경망에 4차원 데이터가 하나 흐를 때마다, 데이터 N개에 대한 합성곱 연산이 이뤄진다는 것이다. (즉, N회 분의 처리를 한 번에 수행한다는 뜻이다)

## 풀링 계층

풀링(Pooliing)은 세로 * 가로 방향의 공간을 줄이는 연산으로, 일종의 subsampling이 주된 목적이라고 할 수 있다. (즉, 이미지의 크기를 축소하는 것이 목표라는 의미이다)


풀링 방법으로는 "최대 풀링(Max Pooling)"과 "평균 풀링(Average Pooling)"이 있으나, 이미지 인식 분야에서는 주로 최대 풀링(Max Pooling)을 사용한다.

또한 다음과 같이 풀링(Pooling)의 윈도우(Window) 크기와 스트라이드(Stride)는 같은 값으로 설정하는 것이 일반적이다.

<img src='./14.JPG'>

* 풀링 계층의 특징으로는 아래와 같이 3가지를 들 수 있겠다.
1) 학습해야 할 매개변수가 없음
2) 채널 수가 변하지 않음 (입력 데이터의 채널 수 그대로 출력 데이터로 내보냄)
3) 입력 데이터의 변화에 영향을 적게 받음 (즉, 강건함 = Robust)

## 합성곱/풀링 계층 구현하기

- 합성곱 계층과 풀링 계층은 복잡해 보이지만 트릭을 사용하면 쉽게 구현할 수있다.

### 4차원 배열

- 예를 들어 데이터의 형상이 (10, 1, 28, 28)이라면 높이 28, 너비 28, 채널 1개인 데이터가 10개이다.

In [ ]:
>>> x = np.random.rand(10, 1, 28, 28)
>>> x.shape
(10, 1, 28, 28)

>>> x[0].shape # (1, 28, 28), accessing first data
>>> x[0, 0] ## x[0][0], accessing first channel data

### im2col로 데이터 전개하기

1) 합성곱 연산을 곧이곧대로 수행하려면 for문을 엄청 많이 사용해야 한다.
2) for 구문을 사용하면 성능이 떨어진다는 단점도 있다. (numpy에서는 원소 접근 시에 for 문을 사용하지 않는 것이 바람직하다)
3) 이번 절에서는 im2col이라는 편의 함수를 사용해보자
- 입력 데이터를 필터링(가중치 계산)하기 좋게 전개하는 함수이다.

<img src='./15.JPG'>

합성곱 계층의 구현을 3단계로 정리해보면 아래와 같다.

1. 필터(Filter)를 세로로 1열로 전개
2. im2col이 전개한 입력 데이터와 필터의 행렬 곱을 계산
3. 출력 데이터의 형상을 변환(reshape)

### 합성곱 계층 구현하기

* im2col의 인터페이스
- im2col(input_data, filter_h, filter_w, stride=1, pad=0)
- input_data : (데이터 수, 채널 수, 높이, 너비)의 입력데이터
- filter_h : 필터의 높이
- filter_w : 필터의 너비
- stride : 스트라이드
- pad : 패딩

In [7]:
import sys, os
sys.path.append(os.pardir)
import numpy as np

from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride = 1, pad = 0)
print(col1.shape) # (9, 75)

(9, 75)


배치 크기가 1, 채널은 3개, 높이 너비가 7 X 7인 데이터에 대하여 적용하였다.

이제 합성곱 계층을 구현해보자

In [8]:
class Convolution:
  def __init__(self, W, b, stride = 1, pad = 0):
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad
    
  def forward(self, x):
    FN, C, FH, FW = self.W.shape
    N, C, H, W = x.shape
    out_h = int(1 + (H + 2 * self.pad - FH) / self.stride)
    out_w = int(1 + (W + 2 * self.pad - FW) / self.stride)
    
    col = im2col(x, FH, FW, self.stride, self.pad)
    col_W = self.W.reshape(FN, -1).T
    out = np.dot(col, col_W) + self.b
    
    out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
    
    return out

1) 입력 데이터를 im2col로 전개하고 필터도 reshape을 사용해서 2차원 배열로 전개했다. 그리고 이 두개의 행렬의 곱을 구한다.
2) reshpae의 인수를 -1로 지정하였는데, 다차원 배열의 원소 수가 변환 후에도 똑같이 유지되도록 적절히 묶어준다.
3) forward 메소드의 마지막에는 출력 데이터를 적절한 형상으로 바꿔준다.
- 이 때 numpy의 transpose를 사용하여 순서를 바꿔준다.
4) 역전파에서는 im2col을 역으로 처리해야한다. 이 책이 제공하는 col2im 메소드를 사용하여 반대로 수행한다.
- https://github.com/oreilly-japan/deep-learning-from-scratch/blob/master/common/util.py#L71-L99
5) 전체 합성곱 계층의 구현은 https://github.com/oreilly-japan/deep-learning-from-scratch/blob/master/common/layers.py#L198-L243 에 구현되어 있다.

### 풀링 계층 구현하기

1) 합성곱 계층과 마찬가지로 im2col을 사용해서 입력 데이터를 전개한다.
2) 단, 풀링의 경우에는 채널 쪽이 독립적이라는 점이 합성곱 계층 때와 다르다.
- 채널별로 전개한 다음에 해당 행에서 최댓값, 혹은 평균값만을 골라내면 된다.

<img src='./16.JPG'>

In [9]:
class Pooling:
  def __init__(self, pool_h, pool_w, stride = 1, pad = 0):
    self.pool_h = pool_h
    self.pool_w = pool_w,
    self.stride = stride
    self.pad = pad
    
  def forward(self, x):
    N, C, H, W = x.shape
    out_h = int(1 + (H - self.pool_h) / self.stride)
    out_w = int(1 + (W - self.pool_w) / self.stride)
    
    # 전개(1)
    col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
    col = col.reshape(-1, self.pool_h * self.pool_w)
    
    # 전개(1)
    out = np.max(col, axis = 1)
    
    # 전개(3)
    out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

@ 풀링 계층 구현은 3단계로 진행한다.

1) 입력 데이터를 전개
2) 행별 최댓값 구한다
3) 적절한 모양으로 다시 성형한다.

@ 역전파 방향은 ReLU 계층의 max 역전파를 참고하자.
- 코드는 https://github.com/oreilly-japan/deep-learning-from-scratch/blob/master/common/layers.py#L246-L284

## CNN 구현하기

1) conv - relu - pool - affine - relu - affine - softmax 형태
2) 초기화(nit)에서 받는 파라미터(인수)
    input_dim : 입력 데이터(채널 수, 높이, 너비)의 차원
    conv_param : 합성곱 계층의 하이퍼파라미터, 딕셔너리의 키는 다음과 같음
    filter_num : 필터 수
    filter_size : 필터 크기
    stride : 스트라이드
    pad : 패딩
3) hidden_size : 은닉층(완전연결)의 뉴런 수
4) output_size : 출력층(완전연결)의 뉴런 수
5) weight_init_std : 초기화 때의 가중치 표준편차

합성곱계층의 하이퍼파라미터는 딕셔너리 형태로 주어짐

[CNN 구현 및 MNIST 학습코드는 너무 길어서 생략, 책과 데이터셋 참고]

In [ ]:
class SimpleConvNet:
  def __init__(self, input_dim=(1, 28,28), conv_param{'filter_num': 30, 'filter_size': 5,
                                                     'pad': 0, 'stride': 1},
              hidden_size = 100, output_size = 10, weight_init_std = 0.01):
    filter_num = conv_param['filter_num']
    filter_size = conv_param['filter_size']
    filter_pad = conv_param['filter_pad']
    filter_stride = conv_param['filter_stride']
    input_size = input_dim[1]
    conv_output_size = (input_size - filter_size + 2 * filter_pad) / filter_stride + 1
    pool_output_size = int(filter_num * (conv_output_size / 2) * (conv_output_size / 2))
    
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
    self.params['b1'] = np.zeros(filter_num)
    self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
    self.params['b2'] = np.zeros(hidden_size)
    self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b3'] = np.zeros(output_size)
    
    # CNN layer creation
    self.layers = OrderedDict()
    self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                      conv_param['stride'], conv_param['pad'])
    self.layers['Relu1'] = Relu()
    self.layers['Pool1'] = Pooling(pool_h = 2, pool_w = 2, stride = 2)
    self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
    self.layers['Relu2'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
    self.last_layer = SoftmaxWithLoss()
    
  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x)
    return x
  
  def loss(self, x, t):
    y = self.predict(x)
    return self.last_layer.forward(y, t)
  
  def gradient(self, x, t):
    self.loss(x, t)
    
    dout = 1
    dout = self.last_layer.backward(dout)
    
    layers = list(self.layers.values())
    layers.reverse()
    
    for layer in layers:
      dout = layer.backward(dout)
      
    grads = {}
    grads['W1'] = self.layers['Conv1'].dW
    grads['b1'] = self.layers['Conv1'].db
    grads['W2'] = self.layers['Affine1'].dW
    grads['b2'] = self.layers['Affine1'].db
    grads['W3'] = self.layers['Affine2'].dW
    grads['b3'] = self.layers['Affine2'].db
    
    return grads

해당 네트워크로 데이터셋을 학습하면 훈련 데이터에 대한 정확도는 99.82%, 시험 데이터에 대한 정확도는 약 98.96 % 가 된다.

## CNN 시각화하기

CNN을 시각화해보면, 합성곱 계층을 여러 겹 쌓으면 층이 깊어지면서 더 복잡하고 추상화된 정보가 추출된다.
즉, 층이 깊어지면서 뉴런이 반응하는 대상이 단순한 모양에서 "고급" 정보로 변화해간다는 뜻이다.

<img src='./17.JPG'>

## 대표적인 CNN

1) LeNet
- 손글씨 숫자를 인식하는 네트워크로 1998년에 제안됨
- 합성곱 계층과 풀링 계층(여기서는 단순한 subsampling 계층. 즉, 최대 풀링 사용 x)을 반복하고, 마지막에 완전연결 계층을 거치면서 결과를 출력함
- 활성화 함수로 Sigmoid 함수를 사용함

<img src='./18.JPG'>

2) AlexNet
- LeNet과 비교하면 훨씬 최근인 2012년에 발표된 네트워크임
- 기본적인 구성은 LeNet과 크게 다르지 않으나, 다음과 같은 변화를 주었음
    - 활성화 함수로 ReLU 함수를 사용함
    - Local Response Normalization이라는 국소적 정규화를 실시하는 계층을 이용함
    - 드롭아웃(Dropout)을 사용함
    
<img src='./19.JPG'>